# 1. Library 설치

- 자바 설치 필요 (1.8 jdk version)
- mac인 경우 특정 jdk설치 필요 (참고: https://jongsky.tistory.com/22)

In [1]:
#!pip install konlpy
#!pip install jpype1
#!pip install pandas
#!pip install keybert
#!pip install Word2Vec
#!pip install gensim
#!pip install transformers

In [15]:
from konlpy.tag import Okt
import pandas as pd
import json
import re

okt =Okt()
df = pd.read_json('crawling_data_0504.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6406 entries, 0 to 6405
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          6406 non-null   object
 1   news_agency    6406 non-null   object
 2   origin_url     6406 non-null   object
 3   tag            6406 non-null   object
 4   created_date   6406 non-null   object
 5   reporter       6406 non-null   object
 6   image          6406 non-null   object
 7   news_contents  6406 non-null   object
dtypes: object(8)
memory usage: 400.5+ KB


# 2. Okt (형태소 분석)
- 명사, 형용사, 동사 등의 태그를 붙여 원하는 형태소만 추출
- 뉴스에서 의미있는 키워드를 가지고 오기위해 명사만 추출

In [16]:
def toknizer(sentence):
    try:
        morphs = okt.pos(sentence)
        primaryTok = ["Noun"]
        morphs = (morp for morp in morphs if morp[1] in primaryTok)
        morphs = (morph for morph in morphs if not (len(morph[0]) == 1))
        result=''
        for morph in morphs:
            if result!='':
                    result+=" "
            result+=morph[0]
    except Exception as e:
        print('Error:',e)
        print('Error:',sentence)
        result = ''
    return result

In [17]:
# toknizing 진행
df_tok = df['news_contents'].map(toknizer)

In [18]:
print(df['news_contents'][0])
print("----------------------------------------------------------")
print("------------------------toknizing-------------------------")
print("----------------------------------------------------------")
print(df_tok[0])

"하루빨리 통과 자치분권 속도 내야"
"하루빨리 통과 자치분권 속도 내야" 전국 17개 시ㆍ도지사들이 2일 ‘자치분권 및 균형발전 특별법’의 조기 제정을 촉구하고 나섰다.

대한민국시도지사협의회(회장 이철우 경북지사)는 2일 ‘지방시대’ 구현을 위한 제도적 기반이 될 ‘지방자치분권 및 지역균형발전에 관한 특별법(이하 특별법)’의 제정은 더 이상 지체해선 안되는 시급한 과제라면서 정치권에 이같이 요구했다.

이와 함께 일부 교육계에서 우려하는 ‘특별법’의 제35조ㆍ제36조에 대해 “교육자치와 지방자치 ‘통합’의 의미는 행ㆍ재정적 비효율성을 개선하고, 시ㆍ도와 시ㆍ도 교육청간의 연계ㆍ협력을 강화시켜야 한다는 의미로 해석해야 하며 교육의 정치적 중립성ㆍ전문성ㆍ자주성을 저해한다고 단정해서는 안된다”고 강조했다.

또한 “해당 조항은 2010년 ‘행정체제 개편에 관한 특별법’에 신설돼 이미 지난 13년 동안 유지된 조항”이라며 “시ㆍ도와 시ㆍ도 교육청이 분리 운영돼 나타나는 비효율을 개선하고자 하는 의미를 담고 있다”고 역설했다.

이철우 회장은 “자치분권 및 균형발전에 관한 특별법이 하루라도 빨리 통과돼 ‘지방시대위원회’를 중심으로 정부의 자치분권과 균형발전이 속도감 있게 추진되어야 한다”면서 “지방을 살리겠다는 이 법의 취지를 모두가 공감할 수 있도록 문제를 제기하는 국회 법사위원들을 대상으로 설득해 나가겠다”고 밝혔다. 남궁창성
----------------------------------------------------------
------------------------toknizing-------------------------
----------------------------------------------------------
통과 자치 분권 속도 내야 통과 자치 분권 속도 내야 전국 도지사 자치 분권 균형발전 특별법 조기 제정 촉구 대한민국 도지사 협의 회장 이철우 경북 지사 지방시 구현 제도 기반 지방자치 분권 지역 균형발전 특별법 이하 특별법 제

# 3. KeyBERT (키워드 추출)
- <b>KeyBERT란
    - 토픽 모델링 중 키워드 추출을 위해 BERT를 적용한 오픈소스 파이썬 모듈
    - 텍스트 임베딩 단계에서 BERT를 사용하기 때문에 BERT-based모델
    - 원리: BERT를 이용해 문서 레벨에서 주제를 파악하고, N-gram을 위해 단어를 임베딩한다. <br>이후 코사인 유사도를 계산하여 어떤 N-gram또는 단어가 문서와 가장 유사한지 찾아낸다. <br>가장 유사한 단어들은 문서를 가장 잘 설명할 수 있는 키워드로 분류한다.

- <B>mmr
    - 텍스트 요약 작업에서 중복을 최소화 하고 결과의 다양성을 극대화 하는 방법
    - 참고 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf)
    - 가장 유사한 키워드 선택 -> 다음 문서와 유사하지만 이미 선택된 키워드와는 유사하지 않은 새로운 후보를 반복적으로 선택함
    - diversity가 높을수록 유사도가 낮은 키워드가 뽑힘

In [19]:
# 모델 불러올 때 kernel 죽을 경우 Memory 설정 변경 필요
# (mac기준) 터미널에서 jupyter notebook --generate-config 입력 후 주피터 프로파일 생성
# 아나콘다 기본 경로 기준 username/.jupyer/jupyer_noteook_config.py 파일 변경해야함
# c.NotebookApp.max_buffer_size= 100000000000000
# 이 방법으로도 안되면 python3.7 버전 설치
from keybert import KeyBERT
from transformers import BertModel

tr_model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(tr_model)

In [20]:
def key_extraction(sentence):
    keybert = kw_model.extract_keywords(sentence,top_n =20,use_mmr=True, diversity = 0.3)
    result = [k for k,v in keybert]
    return result

In [ ]:
# key_bert 진행
df_tok_keyword = df_tok.map(key_extraction)

In [43]:
#예시
kw_model.extract_keywords(df_tok[0], top_n = 20, use_mmr=True, diversity = 0.3)

[('정부', 0.546),
 ('지방자치', 0.5293),
 ('정치권', 0.5274),
 ('교육청', 0.5101),
 ('대한민국', 0.4724),
 ('국회', 0.4721),
 ('행정', 0.4717),
 ('경북', 0.4609),
 ('위원회', 0.4509),
 ('교육', 0.4213),
 ('전국', 0.4016),
 ('특별법', 0.3898),
 ('제도', 0.3832),
 ('중립성', 0.3577),
 ('협력', 0.3562),
 ('통합', 0.3474),
 ('구현', 0.3178),
 ('중심', 0.3095),
 ('균형발전', 0.309),
 ('효율', 0.2799)]

# 4. Word2Vec (연관 단어 추출)
- 단어간의 유사도를 구할 수 있는 인공신경망 모델 (딥러닝 x) (참고:https://wikidocs.net/22660)

- model학습 parameter

    - CBOW = 중간 단어 예측, skip gram = 중간 단어로 주변 단어 예측

    - CBOW 같은 경우 한 문장에 대해서 한 단어는 1번 학습되지만

    - skip gram 같은 경우 단어 별로 여러번 학습 된다.

    - vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.

    - window = 컨텍스트 윈도우 크기

    - min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)

    - workers = 학습을 위한 프로세스 수

    - sg = 0은 CBOW, 1은 Skip-gram

In [47]:
from gensim.models import Word2Vec, KeyedVectors
import gensim

In [50]:
df_tok_list = list(df_tok_keyword)
len(df_tok_list)


630

In [140]:
df_tok_list = list(df2['keyword'])

In [141]:
# Word2Vec 모델 학습
model = Word2Vec(df_tok_list, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [142]:
# 결과 예시
model.wv.most_similar('교육')

# 키 목록
#model.wv.index_to_key

[('학습', 0.9841373562812805),
 ('교육감', 0.9826186299324036),
 ('학부모', 0.9717382192611694),
 ('학과', 0.9647219181060791),
 ('교육청', 0.9617974758148193),
 ('교육과정', 0.956384539604187),
 ('교실', 0.9554803967475891),
 ('고등학교', 0.9536389112472534),
 ('중학생', 0.9521279335021973),
 ('진학', 0.9481607675552368)]

In [ ]:
# 모델 저장
model.save('word2vec_model')


In [ ]:
# 모델 불러오기
model = Word2Vec.load("word2vec_model")